<a href="https://colab.research.google.com/github/Abdelmounim1996/Distributed-Learning-of-Finite-Gaussian-Mixtures/blob/main/KL_distance_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.linalg import det
from math import sqrt

from joblib import Parallel, delayed

from scipy.linalg import inv , pinv
from scipy import linalg
np.seterr(all='warn')
import time 

In [ ]:
def generate_params(n_components , d ):
  Means = np.random.uniform(0., 10 , (n_components , d))
  Covars = np.random.normal(0.1 , 2 , (n_components , d, d))
  Covars = 1/2 *(Covars.transpose((0, 2, 1)) + Covars)
  Covars.reshape(n_components , -1)[:,::d+1]+= 100
  return [Means , Covars] 

In [ ]:
n_clusters_1 = 10
n_clusters_2 = 10 
dim = 784  
G1 = generate_params(n_clusters_1 , dim )
G2 = generate_params(n_clusters_2 , dim )

In [ ]:
Means_1 , Covars_1 = G1
Means_2 , Covars_2 = G2

In [ ]:
Covars_1.shape, Covars_2.shape

((10, 784, 784), (10, 784, 784))

Vectorization numpy

In [ ]:
pip install memory_profiler

  Created wheel for memory-profiler: filename=memory_profiler-0.58.0-py3-none-any.whl size=30190 sha256=d83f651a17f686bc13c4b24a01571c1437daaf824c60319997edccb9922e43fc
  Stored in directory: /root/.cache/pip/wheels/56/19/d5/8cad06661aec65a04a0d6785b1a5ad035cb645b1772a4a0882
Successfully built memory-profiler


In [ ]:
%load_ext memory_profiler

In [ ]:
%%time 
%memit np.linalg.pinv(Covars_1)
# advantage work with singular array disadvantage very expensive 

peak memory: 459.18 MiB, increment: 192.21 MiB
CPU times: user 7.02 s, sys: 1.59 s, total: 8.6 s
Wall time: 4.61 s


In [ ]:
%%time 
%memit np.linalg.inv(Covars_1)
# disadvantage Not work with singular array advantage speed with inversible matrix 

peak memory: 460.13 MiB, increment: 0.93 MiB
CPU times: user 1.07 s, sys: 300 ms, total: 1.38 s
Wall time: 805 ms


In [ ]:
%%time
%memit np.linalg.det(Covars_1)

/usr/local/lib/python3.7/dist-packages/numpy/linalg/linalg.py:2159: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)


peak memory: 460.47 MiB, increment: 0.34 MiB
CPU times: user 637 ms, sys: 333 ms, total: 969 ms
Wall time: 615 ms


# Loop numpy 

In [ ]:
%%time 
%memit np.array([np.linalg.pinv(mat) for mat in Covars_1 ])

peak memory: 460.50 MiB, increment: 0.03 MiB
CPU times: user 7.13 s, sys: 1.61 s, total: 8.74 s
Wall time: 4.61 s


In [ ]:
%%time 
%memit np.array([np.linalg.inv(mat) for mat in Covars_1 ])

peak memory: 460.52 MiB, increment: 0.07 MiB
CPU times: user 1.12 s, sys: 380 ms, total: 1.5 s
Wall time: 893 ms


# parallel computing



In [ ]:
%%time 
%memit  Parallel(n_jobs=-1)(delayed( np.linalg.pinv)(mat) for mat in Covars_1 )

peak memory: 460.82 MiB, increment: 0.30 MiB
CPU times: user 205 ms, sys: 191 ms, total: 396 ms
Wall time: 4.51 s


In [ ]:
%%time 
%memit Parallel(n_jobs=-1)(delayed( np.linalg.inv)(mat) for mat in Covars_1 )

peak memory: 460.82 MiB, increment: 0.00 MiB
CPU times: user 131 ms, sys: 99.2 ms, total: 230 ms
Wall time: 973 ms


# SCIPY performance 

In [ ]:
%%time 
%memit  np.array([inv(mat) for mat in Covars_1 ])

peak memory: 465.05 MiB, increment: 4.22 MiB
CPU times: user 1.02 s, sys: 325 ms, total: 1.34 s
Wall time: 850 ms


for symetric defini positive matrix 

In [ ]:
def compute_precision_cholesky(covariances):
    n_components, n_features, n_features = covariances.shape
    precisions_chol = np.empty((n_components, n_features, n_features))
    for k, covariance in enumerate(covariances):
        try:
            cov_chol = linalg.cholesky(covariance, lower=True)
            inv =linalg.solve_triangular(cov_chol, np.eye(n_features), lower=True).T
        except linalg.LinAlgError: 
             raise ValueError("chol_not work in array n_: ", k)
        precisions_chol[k] = inv
    return precisions_chol
def det_chol(covariances , chol = False):
  n_components , n_features , n_features =  covariances.shape
  precisions_chol = np.empty((n_components, n_features, n_features))
  if chol :
    for k, covariance in enumerate(covariances):
        try:
            cov_chol = linalg.cholesky(covariance, lower=True)
        except linalg.LinAlgError: 
             raise ValueError("chol_not work in array n_: ", k)   
        precisions_chol[k] = cov_chol
  else :
     precisions_chol = covariances 
  return np.multiply.reduce(precisions_chol.reshape( n_components , -1)[:,::n_features+1 ] , axis=1)

In [ ]:
%%time
%memit chol = compute_precision_cholesky(Covars_1)

peak memory: 465.66 MiB, increment: 0.61 MiB
CPU times: user 657 ms, sys: 260 ms, total: 917 ms
Wall time: 632 ms


In [ ]:
%%time 
%memit np.multiply.reduce(chol.reshape(80 , -1)[:,::785] , axis=1)

peak memory: 465.79 MiB, increment: 0.12 MiB
CPU times: user 92.6 ms, sys: 113 ms, total: 206 ms
Wall time: 262 ms


# Performance KL distance

In [ ]:
# first implementation Vectorized numpy code 

In [ ]:
def KL_Vectorized(G1 , G2):

  Means_1 , Covars_1  =G1
  Means_2 , Covars_2  =G1

  Covars2_det = np.abs(np.linalg.det( Covars_2))
  Covars2_inv = np.linalg.inv(Covars_2)
  Covars1_det = np.abs(np.linalg.det(Covars_1))

  n_features = Means_1.shape[1] ; dis_lst = []
  it = 0
  for  det ,Means  , Inv_2 in  zip( Covars2_det, Means_2 , Covars2_inv ):
    it+= 1
    start = time.time()
    Mu = Means_1 -Means
    dis_lst.append(\
                   0.5*(np.log(det/Covars1_det)+ np.trace(np.dot( Covars_1 , Inv_2 ) , axis1 = 1 , axis2 = 2 )+ np.einsum('ij,ik,kj->i', Mu, Mu, Inv_2) -n_features  )\
                   )
    end = time.time()
    print("iteration : " , it , "take : ", end-start, "s ")
  return np.asarray(dis_lst).T

  

In [ ]:
%%time
%memit KL_Vectorized(G1 , G2 )

/usr/local/lib/python3.7/dist-packages/numpy/linalg/linalg.py:2159: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


iteration :  1 take :  5.057668924331665 s 
iteration :  2 take :  4.674783229827881 s 
iteration :  3 take :  4.79826283454895 s 
iteration :  4 take :  4.773270606994629 s 
iteration :  5 take :  4.9354047775268555 s 
iteration :  6 take :  4.698847532272339 s 
iteration :  7 take :  4.677373886108398 s 
iteration :  8 take :  4.762279510498047 s 
iteration :  9 take :  4.805227756500244 s 
iteration :  10 take :  5.424356460571289 s 
peak memory: 465.87 MiB, increment: 0.08 MiB
CPU times: user 50.2 s, sys: 610 ms, total: 50.8 s
Wall time: 49.8 s


# Nested loops KL distance 

In [ ]:
def Gaussian_distance(mu1, mu2, Sigma1, Sigma2):
  d = mu1.shape[0]
  Sigma2_inv = np.linalg.inv(Sigma2)
  log_det = -(np.log(np.linalg.eigvals(Sigma2_inv)).sum() + np.log(np.linalg.eigvals(Sigma1)).sum())
  trace = np.matrix.trace(Sigma2_inv.dot(Sigma1))
  quadratic_term = (mu2 - mu1).T.dot(Sigma2_inv).dot(mu2 - mu1)
  return .5 * (log_det + trace + quadratic_term - d)

In [ ]:
%%time 
ls=[]
it = 0
for mu1, Sigma1 in zip(Means_1, Covars_1):
  start = time.time()
  ls.append( [Gaussian_distance(mu1, mu2, Sigma1, Sigma2) for mu2, Sigma2 in zip(Means_2, Covars_2) ] )
  it+=1
  end = time.time()
  print("iteration : " , it , "take : ", end-start, "s ")


iteration :  1 take :  15.200534105300903 s 
iteration :  2 take :  15.050174474716187 s 
iteration :  3 take :  15.105452299118042 s 
iteration :  4 take :  15.172537803649902 s 
iteration :  5 take :  15.014572620391846 s 
iteration :  6 take :  15.364394426345825 s 
iteration :  7 take :  14.86424446105957 s 
iteration :  8 take :  15.486286878585815 s 
iteration :  9 take :  15.275287866592407 s 
iteration :  10 take :  15.112645149230957 s 
CPU times: user 3min 21s, sys: 1min 37s, total: 4min 58s
Wall time: 2min 31s


In [ ]:
def KL_cholskey(G1 , G2):

  Means_1 , Covars_1  =G1
  Means_2 , Covars_2  =G1

  Covars2_inv = compute_precision_cholesky(Covars_2)
  Covars2_det = 1/np.abs(det_chol(Covars_2))
  Covars1_det = np.abs(det_chol(Covars_1 , chol= True))

  n_features = Means_1.shape[1] ; dis_lst = []
  it = 0
  for  det ,Means  , Inv_2 in  zip( Covars2_det, Means_2 , Covars2_inv ):
    it+= 1
    start = time.time()
    Mu = Means_1 -Means
    dis_lst.append(\
                   0.5*(np.log(det/Covars1_det)+ np.trace(np.dot( Covars_1 , Inv_2 ) , axis1 = 1 , axis2 = 2 )+ np.einsum('ij,ik,kj->i', Mu, Mu, Inv_2) -n_features  )\
                   )
    end = time.time()
    print("iteration : " , it , "take : ", end-start, "s ")
  return np.asarray(dis_lst).T

In [ ]:
%%time
%memit KL_cholskey(G1 , G2 )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: RuntimeWarning: overflow encountered in reduce
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()


iteration :  1 take :  5.255194664001465 s 
iteration :  2 take :  4.783926963806152 s 
iteration :  3 take :  4.795875549316406 s 
iteration :  4 take :  4.721572160720825 s 
iteration :  5 take :  4.623763561248779 s 
iteration :  6 take :  4.752767324447632 s 
iteration :  7 take :  4.571815729141235 s 
iteration :  8 take :  4.7560200691223145 s 
iteration :  9 take :  4.731496810913086 s 
iteration :  10 take :  5.2347612380981445 s 
peak memory: 466.28 MiB, increment: 0.04 MiB
CPU times: user 49.1 s, sys: 536 ms, total: 49.6 s
Wall time: 49 s


In [ ]:
def KL_cholskey(G1 , G2):

  Means_1 , Covars_1  = G1
  Means_2 , Covars_2  = G1

  Covars2_inv = compute_precision_cholesky(Covars_2)
  Covars2_det = 1/np.abs(det_chol(Covars_2))
  Covars1_det = np.abs(det_chol(Covars_1 , chol= True))

  n_features = Means_1.shape[1] ; dis_lst = []
  it = 0
  for  det ,Means  , Inv_2 in  zip( Covars2_det, Means_2 , Covars2_inv ):
    it+= 1
    start = time.time()
    Mu = Means_1 -Means
    dis_lst.append(\
                   0.5*(np.log(det/Covars1_det)+ np.trace(np.dot( Covars_1 , Inv_2 ) , axis1 = 1 , axis2 = 2 )+ np.sum(np.linalg.solve(np.linalg.cholesky(Inv_2), Mu.T)**2, axis=0) -n_features  )\
                   )
    end = time.time()
    print("iteration : " , it , "take : ", end-start, "s ")
  return np.asarray(dis_lst).T

In [ ]:
%%time
%memit args = KL_cholskey(G1 , G2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: RuntimeWarning: overflow encountered in reduce
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()


iteration :  1 take :  5.259395122528076 s 
iteration :  2 take :  4.74971866607666 s 
iteration :  3 take :  4.611987590789795 s 
iteration :  4 take :  4.961650609970093 s 
iteration :  5 take :  4.706448078155518 s 
iteration :  6 take :  4.6860740184783936 s 
iteration :  7 take :  4.717453956604004 s 
iteration :  8 take :  4.890672445297241 s 
iteration :  9 take :  4.902123928070068 s 
iteration :  10 take :  5.180621862411499 s 
peak memory: 466.39 MiB, increment: 0.10 MiB
CPU times: user 50 s, sys: 1.6 s, total: 51.6 s
Wall time: 49.4 s


In [ ]:
args

array([[-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf]])

In [ ]:
def Parallel_KL(GMM_AVG , GMM_reduce ):
  # stock params 
  Means_AVG , Covars_AVG = GMM_AVG ; GMM_AVG = None 
  Means_reduce , Covars_reduce = GMM_reduce ; GMM_reduce = None 
  # getting shape 
  n_components_AVG ,n_features =  Means_AVG.shape
  n_components_reduce ,n_features =  Means_AVG.shape
  # useful params avg 
  Inv_AVG = Inv_cholesky(Covars_AVG)
  det_AVG = np.multiply.reduce( np.linalg.cholesky(Covars_AVG).reshape(n_components_AVG , -1 )[:, ::n_features+1], axis=1)**2;  Covars_AVG = None
  # useful params reduce 
  det_reduce = np.multiply.reduce( np.linalg.cholesky(Covars_reduce).reshape(n_components_reduce , -1 )[:, ::n_features+1], axis=1)**2
  # parallel loops 4 cores 
  parallel_args = Parallel(n_jobs=-1)(delayed(lambda x, y ,z :
                                              0.5*( np.log(np.abs(x/det_reduce))+\
                                                   np.trace(np.dot(Covars_reduce , S ) , axis1 = 1 , axis2 = 2 )+\
                                                   np.sum(np.linalg.solve(np.linalg.cholesky(S), (Means_reduce - mu ).T)**2, axis=0) -n_features)
                                              )(det, mu , S) for det, mu , S in zip(det_AVG , Means_AVG ,Inv_AVG   ))
  
  return parallel_args
  


def Inv_cholesky(covariances):
    n_components, n_features, n_features = covariances.shape
    precisions_chol = np.empty((n_components, n_features, n_features))
    for k, covariance in enumerate(covariances):
        try:
            cov_chol = linalg.cholesky(covariance, lower=True)
            inv =linalg.solve_triangular(cov_chol, np.eye(n_features), lower=True).T
        except linalg.LinAlgError: 
             raise ValueError("chol_not work in array n_: ", k)
        precisions_chol[k] = inv
    return precisions_chol
def det_chol(covariances , chol = False):
  n_components , n_features , n_features =  covariances.shape
  precisions_chol = np.empty((n_components, n_features, n_features))
  if chol :
    for k, covariance in enumerate(covariances):
        try:
            cov_chol = linalg.cholesky(covariance, lower=True)
        except linalg.LinAlgError: 
             raise ValueError("chol_not work in array n_: ", k)   
        precisions_chol[k] = cov_chol
  else :
     precisions_chol = covariances 
  return np.multiply.reduce(precisions_chol.reshape( n_components , -1)[:,::n_features+1 ] , axis=1)





In [ ]:
%%time
 args = Parallel_KL(G1 , G2 )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in reduce
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: overflow encountered in reduce
  if sys.path[0] == '':


CPU times: user 1.34 s, sys: 510 ms, total: 1.85 s
Wall time: 51.8 s
